In [26]:
import yaml
from typing import List, Dict
from sklearn.model_selection import train_test_split
import uproot
import os
import glob
import xgboost as xgb
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [6]:
import tensorflow as tf

def mlpClassifier(input_df, outputs):
    input_hits = tf.keras.Input(input_df)

    x = tf.keras.layers.Dense(64, activation='relu')(input_hits)
    x = tf.keras.layers.Dense(32, activation='relu')(x)
    output = tf.keras.layers.Dense(n_outputFeatures, activation='sigmoid')(x)

    model = tf.keras.Model(inputs=[input_hits], outputs=[output])
    return model

model = mlpClassifier(window

Model.fit(...)

SyntaxError: '(' was never closed (242134116.py, line 15)

In [29]:
class mlpScikitClassifier:
    def __init__(self, yaml_file: str): # identifying that the yaml file should be a string
        """
        Initialize with configurations from a YAML file.
        :param yaml_file: Path to the YAML configuration file.
        """
        # defining each of the attributes
        self.input_paths = None
        self.samples = {}
        self.variables = []
        self.hyperparameters = {}
        self.model = None
        self.dataframes = {"background": None, "signal": None}

        # calling two of the functions up here, even though only defined below
        # can do this bc reads classes all first, then goes through
        self._parse_yaml(yaml_file)
        self._load_data()

    def _parse_yaml(self, yaml_file: str):
        try: # try means that if error happens, it'll go to exception
            # use 'with' to make sure that yaml file is closed after it's opened
            with open(yaml_file, 'r' ) as file: # telling to read the file and call it "file"
                config = yaml.safe_load(file)

            # defining the attributes from things inside yaml file
            self.input_paths = config['input_path']
            self.samples[0] = config.get(0, []) # defining first column of samples as those under the key 0 in the yaml file - also, if 0 doesn't exist, return empty list "[]"
            self.samples[1] = config.get(1, []) # defining second column of samples to be things under key 1 in file
            self.variables = config.get('variables', []) # defining variables to be things under key 'variables'

        # if error occurs inside the try, will give this formatted error
        except Exception as e:
            raise ValueError(f"Error parsing YAML file: {e}") # f-string that lets you embed variables in string

    def _load_data(self):
        for sample_type, key in [("background", 0), ("signal", 1)]: # iterating this for loop, once for signal, once for background
            sample_list = self.samples[key] # calling all the background/signal samples
            dataframes = []

            for sample in sample_list:
                files = glob.glob(os.path.join(f"{self.input_paths}/{sample}/", "*.root")) # joining the sample with input path and making root file
                for file in files: ## why do we need this for loop, if there should only be one file found for each sample?
                    self.tree = uproot.open(file)['analysis'] # opening root file and ttree
                    variables_with_b_w = self.variables + ['b_w']
                    df = self.tree.arrays(variables_with_b_w, library="pd") # turning tree into pd df
                    df['sample'] = sample # adding column for sample name
                    df['label'] = key # adding column for 0 or 1
                    dataframes.append(df) # adding all background df into one list

            combined_df = pd.concat(dataframes, ignore_index=True).dropna() # concatinating the list of df into one full df
            self.dataframes[sample_type] = combined_df # making the df an attribute, and placing them in dictionary with the key of either signal or background

    def print_config(self):
        """Prints the current configuration."""
        print(f"Input Paths: {self.input_paths}")
        print(f"Samples: {self.samples}")
        print(f"Variables: {self.variables}")

    def prepare_data(self):
        # putting data into sep background and signal df
        self.background_df = self.dataframes["background"]
        self.signal_df = self.dataframes["signal"]

        # computing ratio between sig and back for reweighting when training model
        #self.scale = len(background_df) / len(signal_df)

        self.combined_df = pd.concat([self.background_df, self.signal_df], ignore_index=True) # combining sig and back df into one df
        # splitting df into X and y
        self.X = self.combined_df[self.variables]  # Use only the specified variables
        self.y = self.combined_df['label']
        self.weights = self.combined_df['b_w']

        # returning the train/test split so can be used later in class
        return train_test_split(self.X, self.y, self.weights, test_size=0.3, random_state=42)

    def train_model(self, model_path="xgboost_model.json", importance_plot_path=None): # function parameters with defaults (if don't pass anything when calling function)
        self.X_train, self.X_test, self.y_train, self.y_test, b_w_train, b_w_test = self.prepare_data() # defining X and y train/test

        def create_baseline():
            model = Sequential()
            inputShape = len(self.variables)
            model.add(Dense(64, input_shape=(inputShape,), activation='relu'))
            model.add(Dense(1, activation='sigmoid'))
            model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

            return model

        estimator = KerasClassifier(model=create_baseline, epochs=50, batch_size=10, verbose=1)
        kfold = StratifiedKFold(n_splits=10, shuffle=True)
        results = cross_val_score(estimator, self.X_train, self.y_train, cv=kfold)
        print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

        

In [ ]:
mlp = mlpScikitClassifier('nersc.yaml')
mlp.train_model()

/global/homes/k/kwandr/.local/perlmutter/python-3.11/lib/python3.11/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
W0000 00:00:1752783482.981055  461155 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 1/50
 433402/1525860 ━━━━━━━━━━━━━━━━━━━━ 12:24 681us/step - accuracy: 0.9477 - loss: 2.8239